### Generating reports for invoicing purposes.

In [1]:
import pyodbc
import os
from IPython.core.display import display
from dotenv import load_dotenv
import pandas as pd

Connecting to SQL Server


In [2]:
load_dotenv('credentials.env')

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER='+os.environ.get('server')+';'
    'DATABASE='+os.environ.get('database')+';'
    'UID='+ os.environ.get('uid') +';'
    'PWD='+ os.environ.get('pwd')+';')


#### 1. Retrieve customer orders (store names, sales order ID, store address and the total due).

First 30 rows are selected as an example of the table returned by the query. To see all the rows, the last two lines (OFFSET 0 ROWS FETCH NEXT 30 ROWS ONLY) can be commented.


In [3]:
customer_orders_df = pd.read_sql("SELECT store.Name AS StoreName, "
                                 "header.SalesOrderID, "
                                 "final_adress.AddressLine1, "
                                 "final_adress.City, "
                                 "province.Name AS Region, "
                                 "final_adress.PostalCode, "
                                 "header.TotalDue "
                                 "FROM Sales.Customer AS customer "
                                 "JOIN Sales.Store AS store "
                                 "ON customer.StoreID = store.BusinessEntityID "
                                 "JOIN Sales.SalesOrderHeader AS header "
                                 "ON customer.CustomerID = header.CustomerID "
                                 "JOIN Person.BusinessEntity AS entity "
                                 "ON store.BusinessEntityID = entity.BusinessEntityID "
                                 "JOIN Person.BusinessEntityAddress AS entity_address "
                                 "ON entity.BusinessEntityID = entity_address.BusinessEntityID "
                                 "JOIN Person.Address AS final_adress "
                                 "ON entity_address.BusinessEntityID = final_adress.AddressID "
                                 "JOIN Person.AddressType AS address_type "
                                 "ON entity_address.AddressTypeID = address_type.AddressTypeID "
                                 "JOIN Person.StateProvince AS province "
                                 "ON final_adress.StateProvinceID = province.StateProvinceID "
                                 "ORDER BY StoreName "
                                 "OFFSET 0 ROWS "
                                 "FETCH NEXT 30 ROWS ONLY;", conn)

display(customer_orders_df)

,StoreName,SalesOrderID,AddressLine1,City,Region,PostalCode,TotalDue
0,A Bike Store,43860,2545 N.W. 107th Ave.,Miami,Florida,33127,12381.0798
1,A Bike Store,44501,2545 N.W. 107th Ave.,Miami,Florida,33127,22152.2446
2,A Bike Store,45283,2545 N.W. 107th Ave.,Miami,Florida,33127,31972.1684
3,A Bike Store,46042,2545 N.W. 107th Ave.,Miami,Florida,33127,29418.5269
4,Acclaimed Bicycle Company,53618,5 Madrid,Concord,California,94519,3931.9148
5,Acclaimed Bicycle Company,59070,5 Madrid,Concord,California,94519,2390.2882
6,Acclaimed Bicycle Company,65259,5 Madrid,Concord,California,94519,1322.2843
7,Acclaimed Bicycle Company,71883,5 Madrid,Concord,California,94519,680.3498
8,Ace Bicycle Supply,46982,City Center,Minneapolis,Minnesota,55402,74.0972
9,Ace Bicycle Supply,48007,City Center,Minneapolis,Minnesota,55402,729.6412



#### 2. Retrieve stores' names, respective salespeople who work there, their full name and contact information


In [4]:
sales_person_contacts_df = pd.read_sql("SELECT DISTINCT(st.Name) AS StoreName, "
                                    "p.FirstName + ' ' + ISNULL(MiddleName + ' ', '') + p.LastName as FullName, "
                                    "em.EmailAddress, ph.PhoneNumber "
                                    "FROM Sales.Customer AS c "
                                    "JOIN Sales.Store AS st "
                                    "ON c.StoreID = st.BusinessEntityID "
                                    "JOIN Sales.SalesPerson AS sp "
                                    "ON st.SalesPersonID = sp.BusinessEntityID "
                                    "JOIN HumanResources.Employee AS e "
                                    "ON sp.BusinessEntityID = e.BusinessEntityID "
                                    "JOIN Person.Person AS p "
                                    "ON e.BusinessEntityID = p.BusinessEntityID "
                                    "LEFT JOIN Person.EmailAddress AS em "
                                    "ON p.BusinessEntityID = em.BusinessEntityID "
                                    "LEFT JOIN Person.PersonPhone as ph "
                                    "ON p.BusinessEntityID = ph.BusinessEntityID "
                                    "ORDER BY st.Name;", conn)

display(sales_person_contacts_df)

,StoreName,FullName,EmailAddress,PhoneNumber
0,A Bicycle Association,Michael G Blythe,michael9@adventure-works.com,257-555-0154
1,A Bike Store,Pamela O Ansman-Wolfe,pamela0@adventure-works.com,340-555-0193
2,A Cycle Shop,David R Campbell,david8@adventure-works.com,740-555-0182
3,A Great Bicycle Company,Jillian Carson,jillian0@adventure-works.com,517-555-0117
4,A Typical Bike Shop,Jillian Carson,jillian0@adventure-works.com,517-555-0117
...,...,...,...,...
696,World Bike Discount Store,Linda C Mitchell,linda3@adventure-works.com,883-555-0116
697,World of Bikes,Jillian Carson,jillian0@adventure-works.com,517-555-0117
698,Worthwhile Activity Store,Tsvi Michael Reiter,tsvi0@adventure-works.com,664-555-0112
699,Year-Round Sports,David R Campbell,david8@adventure-works.com,740-555-0182



#### 3. Retrieve the availability of sales quota for sales people


In [5]:
sales_quota_df = pd.read_sql("SELECT p.FirstName + ' ' + ISNULL(MiddleName + ' ', '') + p.LastName as FullName, "
                             "e.JobTitle, "
                             "sp.SalesQuota, "
                             "sp.Bonus, "
                             "sp.CommissionPct, "
                             "sp.SalesYTD, "
                             "CASE WHEN SalesQuota IS NULL THEN 'No Specified Quota' "
                             "ELSE 'Quota Available' END AS QuotaAvailability "
                             "FROM Sales.SalesPerson sp "
                             "INNER JOIN HumanResources.Employee e "
                             "ON sp.BusinessEntityID = e.BusinessEntityID "
                             "INNER JOIN Person.Person p "
                             "ON e.BusinessEntityID = p.BusinessEntityID "
                             "ORDER BY FullName;", conn)

display(sales_quota_df)

,FullName,JobTitle,SalesQuota,Bonus,CommissionPct,SalesYTD,QuotaAvailability
0,Amy E Alberts,European Sales Manager,NaN,0.0,0.000,5.199059e+05,No Specified Quota
1,David R Campbell,Sales Representative,250000.0,3500.0,0.012,1.573013e+06,Quota Available
2,Garrett R Vargas,Sales Representative,250000.0,500.0,0.010,1.453719e+06,Quota Available
3,Jae B Pak,Sales Representative,250000.0,5150.0,0.020,4.116871e+06,Quota Available
4,Jillian Carson,Sales Representative,250000.0,2500.0,0.015,3.189418e+06,Quota Available
5,José Edvaldo Saraiva,Sales Representative,250000.0,5000.0,0.015,2.604541e+06,Quota Available
6,Linda C Mitchell,Sales Representative,250000.0,2000.0,0.015,4.251369e+06,Quota Available
7,Lynn N Tsoflias,Sales Representative,250000.0,5650.0,0.018,1.421811e+06,Quota Available
8,Michael G Blythe,Sales Representative,300000.0,4100.0,0.012,3.763178e+06,Quota Available
9,Pamela O Ansman-Wolfe,Sales Representative,250000.0,5000.0,0.010,1.352577e+06,Quota Available
